In [3]:
import subprocess

In [5]:
def run_cql_command(command):
    process = subprocess.Popen(
        ['cqlsh', '-e', command],
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE
    )
    stdout, stderr = process.communicate()
    
    if process.returncode != 0:
        print(f"Error: {stderr.decode()}")
    else:
        print(stdout.decode())

In [6]:
create_keyspace = """
CREATE KEYSPACE IF NOT EXISTS testkeyspace
WITH replication = {'class': 'SimpleStrategy', 'replication_factor' : 1};
"""

In [7]:
run_cql_command(create_keyspace)

FileNotFoundError: [Errno 2] No such file or directory: 'cqlsh'